In [1]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# ユーザが与えたトピックについて短いジョークをLLMに言わせるためのchain
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}") # プロンプトのテンプレート(invoke()を持っていて、このままchainに組み込めるところがふつうのstrと違う)
model = ChatOpenAI(model="gpt-3.5-turbo") # モデルにアクセスするためのAPI(モデルとの通信とかもこいつがやってそう)
output_parser = StrOutputParser() # モデルから帰ってきた出力をAIMessageからstrにしてくれるやつ、そしてchainに組み込めるやつ

chain = prompt | model | output_parser # これでchainを書けるの、イケてると思う

# chain.invoke({"topic": "ice cream"})

# 1. Prompt

プロンプトは`BasePromptTemplate`であり、テンプレート変数の辞書を受け取って`PromptValue`を返す。  
`PromptValue`は完成したプロンプトのラッパーで、`LLM`(strを受け取る)や`ChatModel`(メッセージのシーケンスを受け取る)に入力することができる。

プロンプトはstring出力と`BaseMessage`のシーケンスのどちらも出力するロジックを持っているので、両方のモデルと組み合わせることができる。

めっちゃ大事なこと書いてあった。

In [2]:
prompt_value = prompt.invoke({"topic": "ice cream"}) # `BasePromptTemplate`を`invoke()`すると`PromptValue`が返される
prompt_value # PromptValue形式になっていることがわかる

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [3]:
prompt_value.to_messages() # `PromptValue`から`BaseMessage`を生成するロジック

[HumanMessage(content='tell me a short joke about ice cream')]

In [4]:
prompt_value.to_string() # `PromptValue`からstrを生成するロジック

'Human: tell me a short joke about ice cream'

# 2. Model

`PromptValue`は`model`に渡される。このケースでは`ChatModel`であり、`BaseMessage`のシーケンスを受け取る。`ChatModel`の出力も`BaseMessage`になる。

In [5]:
# message = model.invoke(prompt_value) # PromptValueを受け取ってBaseMessageをかえす
# message

In [6]:
# modelが`LLM`なら出力はstr
from langchain_openai.llms import OpenAI

# llm = OpenAI(model="gpt-3.5-turbo-instruct") # `ChatModel`を呼び出すときは`ChatOpenAI()`, `LLM`を呼び出すときは`OpenAI()`
# llm.invoke(prompt_value) # チュートリアルと比べて、行頭のRobot:がないので、少し仕様が変わっていそう

# 3. Output parser

`model`の出力を`output_parser`に渡す。これは`BaseOutputParser`で、stringか`BaseMessage`を入力に受け取る。`StrOutputParser`は特別にどんな入力もstringに変える。

In [7]:
# output_parser.invoke(message) # 今回のは`StrOutputParser`なので、すべての入力はstrになる

# 4. パイプライン全体

1. ユーザが所望のトピックを`{"topic": "ice cream"}`として渡す。
1. `prompt`がユーザの入力を受け取り、`PromptValue`を出力する
1. `model`が`PromptValue`を受け取り、OpenAIのLLM modelに渡して出力を生成してもらう。モデルの出力は`ChatMessage`
1. `output_parser`が`ChatMessage`を受け取り、Pythonの文字列に変換する。

In [8]:
input = {"topic": "ice cream"}

prompt.invoke(input) # ChatPromptValueを返す
# > ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

ChatPromptValue(messages=[HumanMessage(content='tell me a short joke about ice cream')])

In [9]:
# (prompt | model).invoke(input) # promptがChatPromptValueを返し、それがmodel(ChatModel, BaseMessageのシーケンスを受け取る)に入力され、AIMessageを返す
# > AIMessage(content="Why did the ice cream go to therapy?\nBecause it had too many toppings and couldn't cone-trol itself!")


In [19]:
# Requires:
# pip install langchain docarray tiktoken

from langchain_community.vectorstores import DocArrayInMemorySearch # ドキュメントを保存しておくデータベースである、とだけ理解
from langchain_core.output_parsers import StrOutputParser # モデルの出力した`BaseMessage`をstringに直す
from langchain_core.prompts import ChatPromptTemplate # invoke()することでモデルに渡せるPromptValueを吐き出すプロンプト
from langchain_core.runnables import RunnableParallel, RunnablePassthrough # まだわからない
from langchain_openai.chat_models import ChatOpenAI # ChatModel(BaseMessageを返すモデル)
from langchain_openai.embeddings import OpenAIEmbeddings # データベースに保存する文書ベクトルは、OpenAIのモデルで文書を埋め込むことで生成される

# データベースのインスタンス
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"], # 2つの文書をOpenAIのモデルによって埋め込んで検索可能な形で保存
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever() 

# ChatPromptTemplateで読み込むためのテンプレートをテキスト形式で定義
# {変数名}としたところの値は、invoke()のときに{変数名: 値}の辞書形式で受け取る
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template) # テンプレートからプロンプトを作る
model = ChatOpenAI() # モデルを呼び出すAPI
output_parser = StrOutputParser() # OutPutParserのインスタンス化

# 
setup_and_retrieval = RunnableParallel( 
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("where did harrison work?")



ValidationError: 2 validation errors for DocArrayDoc
text
  Field required [type=missing, input_value={'embedding': [-0.0192381..., 0.010137099064823456]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing
metadata
  Field required [type=missing, input_value={'embedding': [-0.0192381..., 0.010137099064823456]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing

In [26]:
# データベースのインスタンス
vectorstore = DocArrayInMemorySearch.from_texts(
    texts=["harrison worked at kensho", "bears like to eat honey"], # 2つの文書をOpenAIのモデルによって埋め込んで検索可能な形で保存
    embedding=OpenAIEmbeddings(),
    metadata = [{"source": "Document 1", "author": "Author A"}],
    
)
retriever = vectorstore.as_retriever()

retriever.invoke("where did harrison work?")


ValidationError: 2 validation errors for DocArrayDoc
text
  Field required [type=missing, input_value={'embedding': [-0.0192381..., 0.010137099064823456]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing
metadata
  Field required [type=missing, input_value={'embedding': [-0.0192381..., 0.010137099064823456]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing

In [20]:
# データベースのインスタンス
vectorstore = DocArrayInMemorySearch.from_texts(
    [
        {"text": "harrison worked at kensho", "metadata": {"source": "Document 1", "author": "Author A"}},
        {"text": "bears like to eat honey", "metadata": {"source": "Document 2", "author": "Author B"}}
    ],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

retriever.invoke("where did harrison work?")


TypeError: expected string or buffer

In [18]:
retriever.invoke("where did harrison work?")

ValidationError: 2 validation errors for DocArrayDoc
text
  Field required [type=missing, input_value={'embedding': [-0.0192381..., 0.010137099064823456]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing
metadata
  Field required [type=missing, input_value={'embedding': [-0.0192381..., 0.010137099064823456]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing

In [13]:
type(vectorstore)

langchain_community.vectorstores.docarray.in_memory.DocArrayInMemorySearch

In [14]:
type(retriever)

langchain_core.vectorstores.VectorStoreRetriever

In [23]:
from pydantic import BaseModel, ValidationError


class Model(BaseModel):
    x: str


try:
    Model(x="hello")
except ValidationError as exc:
    print(repr(exc.errors()[0]['type']))
    #> 'missing'
